<img src="images/header.png" alt="Logo UCLM-ESII" align="right">

<br><br><br><br>
<h2><font color="#92002A" size=4>Trabajo Fin de Grado</font></h2>

<h1><font color="#6B001F" size=5>Generación automática de playlist de canciones <br> mediante técnicas de minería de datos</font></h1>
<h2><font color="#92002A" size=3>Parte 0 - Estudio del 'Million Playlist Dataset' (MPD)</font></h2>

<br>
<div style="text-align: right">
    <font color="#B20033" size=3><strong>Autor</strong>: <em>Miguel Ángel Cantero Víllora</em></font><br>
    <br>
    <font color="#B20033" size=3><strong>Directores</strong>: <em>José Antonio Gámez Martín</em></font><br>
    <font color="#B20033" size=3><em>Juan Ángel Aledo Sánchez</em></font><br>
    <br>
<font color="#B20033" size=3>Grado en Ingeniería Informática</font><br>
<font color="#B20033" size=2>Escuela Superior de Ingeniería Informática | Universidad de Castilla-La Mancha</font>

</div>

---

<br>

<a id="indice"></a>
<h2><font color="#92002A" size=5>Índice</font></h2>

<br>

* [1. Introducción](#section1)
* [2. Carga de ficheros](#section2)
* [3. Estructura de los ficheros JSON](#section3)
* [4. Criterios de construcción](#section4)  
* [5. Observaciones](#section5)
* [6. Conclusiones](#section6)

<br>

---

In [1]:
# Permite establecer la anchura de la celda
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
#!pip install emoji
#!pip install regex

In [3]:
import emoji
import json
import os
import regex as re

from collections import Counter
from emoji import UNICODE_EMOJI


---

<br>

<a id="section1"></a>
## <font color="#92002A">1 - Introducción</font>
<br>

Para generar nuestro DataSet con playlists de *Spotify*, hemos tomado como referencia una muestra del conjunto de listas usado para la competición "[__RecSys Challenge 2018__](https://recsys-challenge.spotify.com/)". Dicha muestra está publicada en *GitHub* dentro del proyecto [_Spotify-Song-Recommendation-ML_](https://github.com/vaslnk/Spotify-Song-Recommendation-ML) del usuario *vaslnk*.

A este conjunto se le conoce por el nombre de ["__Million Playlist Dataset__" (MPD)](https://recsys-challenge.spotify.com/readme). El MPD contiene un millón de listas de reproducción (playlists) generadas por usuarios de _Spotify_. Estas listas se crearon durante el periodo de enero de 2010 y octubre de 2017. Cada playlist contiene su título, la lista de canciones (incluyendo metadatos de cada una), información de edición (última vez editada y número de veces editada) y otra información diversa sobre la playlist.

En esta libreta vamos a ver qué características posee dicha muestra, para llevar parte de ellas al conjunto que queremos crear.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section2"></a>
## <font color="#92002A">2 - Carga de ficheros</font>
<br>

In [4]:
# Variables globales

# Directorio donde se encuentra la muestra del MPD
MPD_PATH = 'spotifyMPD/'

In [5]:
mpd_files = [(MPD_PATH + file) for file in os.listdir(MPD_PATH) 
             if file.startswith('mpd.slice.') & file.endswith('.json')]

mpd_dict_list = []

for file in mpd_files:
    with open(file, 'r') as f:
        mpd_dict_list.append(json.load(f))

print(f"{len(mpd_dict_list)} ficheros cargados")

mpd_files

4 ficheros cargados


['spotifyMPD/mpd.slice.1000-1999.json',
 'spotifyMPD/mpd.slice.0-999.json',
 'spotifyMPD/mpd.slice.2000-2999.json',
 'spotifyMPD/mpd.slice.3000-3999.json']

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section3"></a>
## <font color="#92002A">3 - Estructura de los ficheros JSON</font>

<br>

El _Million Playlist Dataset_ está dividido en 1.000 ficheros, donde cada uno contiene 1.000 listas de reproducción. Estos ficheros se han nombrado siguiendo la siguiente convención:

<br>

```
mpd.slice.STARTING_PLAYLIST_ID_-_ENDING_PLAYLIST_ID.json
```

<br>

Por ejemplo, las primeras 1.000 listas del MPD están contenidas en un fichero llamado `mpd.slice.0-999.json` y las últimas 1.000 listas están en un fichero llamado `mpd.slice.999000-999999.json`.

<br>

Cada bloque es un diccionario JSON con dos campos:

- ___info___
- ___playlists___

In [6]:
# Usamos como ejemplo el primer diccionario que tenemos en la lista
example_dict = mpd_dict_list[0]

example_dict.keys()

dict_keys(['info', 'playlists'])

---

<a id="section31"></a>
### <font color="#B20033">El campo '*info*'</font>
<br>

El campo _info_ es un diccionario que contiene información general sobre el bloque en particular:
* ___slice___: Porción de listas que contiene.
* ___version___: Versión del conjunto de datos.
* ***generated_on***: Fecha en la que se ha generado (formato *timestamp*).

In [7]:
example_dict['info']

{'generated_on': '2017-12-03 08:41:42.057563',
 'slice': '1000-1999',
 'version': 'v1'}



---

<a id="section32"></a>
### <font color="#B20033">El campo '*playlists*'</font>
<br>

Este campo es un array que contiene 1.000 listas. Cada lista es un diccionario que posee los siguientes campos:

* ***pid***: [integer] Identificador de la playlist dentro del conjunto. Es un número entre 0 y 999.999
* ***name***: [string] Título de la lista.
* ***modified_at***: [seconds - timestamp] Fecha de modificación.
* ***num_artists***: [integer] Número total de artistas únicos que contiene la playlist.
* ***num_albums***: [integer] Número total de álbumes únicos que contiene la playlist.
* ***num_tracks***: [integer] Número total de pistas que contiene la playlist.
* **num_followers**: [integer] Número de seguidores que tiene la playlist.
* ***num_edits***: [integer] Número de sesiones de edición. Se considera que las pistas agregadas en una ventana de dos horas han sido agregadas en una sola sesión de edición.
* ***duration_ms***: [miliseconds - timestamp] Duración total de la playlist.
* ***collaborative***: [boolean] Si es colaborativa.
* ***tracks***: Lista de diccionarios con información sobre las pistas. Cada diccionario contiene los siguientes campos:
    * *track_name*: Título de la pista.
    * *track_uri*: Dirección Spotify de la pista.
    * *album_name*: Título del álbum.
    * *album_uri*: Dirección Spotify del álbum.
    * *artist_name*: Nombre del artista.
    * *artist_uri*: Dirección Spotify del artista.
    * *duration_ms*: Duración en milisegundos de la pista.
    * *pos*: Posición de la pista dentro de la playlist.    

In [8]:
example_dict['playlists'][0].keys()

dict_keys(['name', 'collaborative', 'pid', 'modified_at', 'num_tracks', 'num_albums', 'num_followers', 'tracks', 'num_edits', 'duration_ms', 'num_artists'])

In [9]:
example_dict['playlists'][0]['tracks'][0].keys()

dict_keys(['pos', 'artist_name', 'track_uri', 'artist_uri', 'track_name', 'album_uri', 'duration_ms', 'album_name'])

---

<a id="section33"></a>
### <font color="#B20033">Ejemplo</font>

<br>

A continuación se muestra un item de ejemplo perteneciente a la lista de playlists:

```json
{
    "name": "musical",
    "collaborative": "false",
    "pid": 5,
    "modified_at": 1493424000,
    "num_albums": 7,
    "num_tracks": 12,
    "num_followers": 1,
    "num_edits": 2,
    "duration_ms": 2657366,
    "num_artists": 6,
    "tracks": [
        {
            "pos": 0,
            "artist_name": "Degiheugi",
            "track_uri": "spotify:track:7vqa3sDmtEaVJ2gcvxtRID",
            "artist_uri": "spotify:artist:3V2paBXEoZIAhfZRJmo2jL",
            "track_name": "Finalement",
            "album_uri": "spotify:album:2KrRMJ9z7Xjoz1Az4O6UML",
            "duration_ms": 166264,
            "album_name": "Dancing Chords and Fireflies"
        },
        {
            "pos": 1,
            "artist_name": "Degiheugi",
            "track_uri": "spotify:track:23EOmJivOZ88WJPUbIPjh6",
            "artist_uri": "spotify:artist:3V2paBXEoZIAhfZRJmo2jL",
            "track_name": "Betty",
            "album_uri": "spotify:album:3lUSlvjUoHNA8IkNTqURqd",
            "duration_ms": 235534,
            "album_name": "Endless Smile"
        },
        {
            "pos": 2,
            "artist_name": "Degiheugi",
            "track_uri": "spotify:track:1vaffTCJxkyqeJY7zF9a55",
            "artist_uri": "spotify:artist:3V2paBXEoZIAhfZRJmo2jL",
            "track_name": "Some Beat in My Head",
            "album_uri": "spotify:album:2KrRMJ9z7Xjoz1Az4O6UML",
            "duration_ms": 268050,
            "album_name": "Dancing Chords and Fireflies"
        },
        // 8 tracks omitted
        {
            "pos": 11,
            "artist_name": "Mo' Horizons",
            "track_uri": "spotify:track:7iwx00eBzeSSSy6xfESyWN",
            "artist_uri": "spotify:artist:3tuX54dqgS8LsGUvNzgrpP",
            "track_name": "Fever 99\u00b0",
            "album_uri": "spotify:album:2Fg1t2tyOSGWkVYHlFfXVf",
            "duration_ms": 364320,
            "album_name": "Come Touch The Sun"
        }
    ],
}
```

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section4"></a>
## <font color="#92002A">4 - Criterios de construcción</font>

<br>

El _Million Playlist Dataset_ se ha creado muestreando listas de reproducción de los miles de millones que existen en _Spotify_ y que los usuarios han creado a lo largo de los años. Las listas de reproducción que cumplen los siguientes criterios, se seleccionan al azar:

* Ha sido creada por un usuario que reside en los Estados Unidos y es mayor de 13 años.
* A la hora en la que fue generado el MPD, la lista era pública.
* Contiene entre 5 y 250 pistas.
* Contiene al menos 3 artistas únicos.
* Contiene al menos 2 álbumes únicos.
* No contiene pistas locales (pistas que no están en _Spotify_ y que el usuario tiene en su dispositivo personal).
* Tienen al menos un seguidor (sin incluir el creador de la playlist).
* Se han creado después del 1 de junio de 2010 y antes del 1 de diciembre de 2017.
* No tiene un título ofensivo.
* No tiene un título orientado a adultos si la lista de reproducción fue creada por un usuario menor de 18 años.

<br>

A continuación, vamos a comprobar algunos de estos criterios en la muestra que hemos obtenido:

In [10]:
playlists = []

# Descartamos la clave 'info' y concatenamos los 
# valores de la clave 'playlists' en una nueva lista
for item in mpd_dict_list:
    playlists += item['playlists']
    
print(f"Número de listas: {len(playlists)} ")

Número de listas: 4000 


---
<a id="section41"></a>
### <font color="#B20033" size=3>Número de pistas</font>
<br>

In [11]:
print("Número máximo de pistas: {0}".format(max([pl['num_tracks'] for pl in playlists])))
print("Número mínimo de pistas: {0}".format(min([pl['num_tracks'] for pl in playlists])))

Número máximo de pistas: 248
Número mínimo de pistas: 5


---
<a id="section42"></a>
### <font color="#B20033" size=3>Número de álbumes</font>
<br>

In [12]:
print("Número mínimo de álbumes: {0}".format(min([pl['num_albums'] for pl in playlists])))

Número mínimo de álbumes: 2


---
<a id="section43"></a>
### <font color="#B20033" size=3>Número de artistas</font>
<br>

In [13]:
print("Número mínimo de artistas: {0}".format(min([pl['num_artists'] for pl in playlists])))

Número mínimo de artistas: 3


---
<a id="section44"></a>
### <font color="#B20033" size=3>Número de seguidores</font>
<br>

In [14]:
print("Número mínimo de seguidores: {0}".format(min([pl['num_followers'] for pl in playlists])))

Número mínimo de seguidores: 1


---

<br>

<div class="alert alert-block alert-warning">

<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
__Importante__: Algunas características como el país de residencia del usuario, fecha de creación o edad del usuario (para comprobar que no contiene títulos orientados a adultos), no es posible obtenerlas ya que la WebAPI de _Spotify_ no proporciona esta información.
</div>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota 1__: Si una lista es pública o contiene pistas locales no podemos comprobarlo en este fragmento del MPD, pero si podremos hacerlo una vez descarguemos las playlists. 
</div>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota 2__:
Podemos comprobar si un título es ofensivo, pero no de forma directa (WebAPI de *Spotify*). Para ello, buscaremos un conjunto de '*palabras ofensivas*' que nos permitan realizar esta comprobación.
</div>

<br>

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section5"></a>
## <font color="#92002A">5 - Observaciones</font>
<br>

A parte de los criterios de construcción con los que se llevó a cabo el DataSet, vamos a hacer algunas comprobaciones en los títulos para ver si podemos extraer alguna nueva característica que nos ayude a crear nuestro propio conjunto.

### <font color="#B20033" size=3>Longitud de los títulos</font>
<br>

In [15]:
# Creamos una lista con los títulos de las playlists
names = [pl['name'] for pl in playlists]

sort_name = min(names, key=len) # Título más corto
long_name = max(names, key=len) # Título más largo

In [16]:
print("Título más corto:")
print(f"\t{sort_name} --> {len(sort_name)} caracteres\n\n")

print("Título más largo:")
print(f"\t{long_name}")
chars = len(long_name) - long_name.count(' ')
print(f"\t- {chars} caracteres (sin espacios)")

Título más corto:
	pr --> 2 caracteres


Título más largo:
	Fifty Shades Of Grey (Original Motion Picture Soundtrack)
	- 50 caracteres (sin espacios)


<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nuevo criterio__: El título contiene entre 2 y 50 caracteres (sin espacios).
</div>

<br>

Vamos a estudiar también el número de palabras que tiene un título:

In [17]:
print("Número máximo de palabras que tiene un título: {0}".format(max([len(pl['name'].split()) for pl in playlists])))

Número máximo de palabras que tiene un título: 9


<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nuevo criterio__: El título contiene menos de 10 palabras.
</div>

---

### <font color="#B20033" size=3>Existencia de emoticonos</font>
<br>

Estudiamos el número de emoticonos/emojis que contienen los títulos de las playlists:

In [18]:
# Comprueba si el carácter es un emoticono.
def is_emoji(s):
    return s in UNICODE_EMOJI['en']

# Comprueba si un texto contiene emoticonos.
def contains_emoji(text):
    return len([c for c in UNICODE_EMOJI['en'] if(c in str(text))]) > 0

# Comprueba si un texto contiene sólo emoticonos.
def all_text_emoji(text):
    return all([(c in UNICODE_EMOJI['en']) for c in str(text)])

# Cuenta los emoticonos que contiene un texto.
def count_text_emoji(text):
    return sum([(c in UNICODE_EMOJI['en']) for c in str(text)])

# Elimina los emoticonos de un texto.
def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', str(text))

In [19]:
max_emojis = max([len(name) for name in names if count_text_emoji(name)])
print(f"Número máximo de emojis que contiene el título: {max_emojis}")

Número máximo de emojis que contiene el título: 10


<br>

Si el título sólo contiene emojis, el número máximo de estos es:

In [20]:
max_only_emojis = max([len(name) for name in names if all_text_emoji(name)])
print(f"Número máximo de emojis que contiene el título: {max_only_emojis}")

Número máximo de emojis que contiene el título: 6


<br>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nuevos criterios__:
* El título contiene como máximo 10 emoticonos, si también contiene texto.
* Si el título está formado únicamente por emoticonos, contiene un número máximo de 4.
</div>

---

### <font color="#B20033" size=3>Alfabeto</font>
<br>

Por último, vamos a comprobar si los títulos de la muestra de playlists contienen algun carácter que no sea perteneciente al alfabeto latino:

In [21]:
# Comprueba que los caracteres de un texto pertenecen
# al alfabeto latino o al conjunto de caracteres comunes
def contains_valid_characters(input):
    """
    :param input: Texto que queremos comprobar.
    :return: Si pertenece o no al alfabeto (boolean).
    """
    text = str(input)
    pattern = re.compile(r'([\p{IsCommon}\p{IsLatin}]+)', re.UNICODE)
    result = re.search(pattern,text)
    is_valid = False
    if len(result) > 0:
        is_valid = len(result[0]) == len(text)
    return is_valid

Con la función que hemos declarado en el bloque anterior, vamos a imprimir aquellos títulos que contienen algún carácter que no sea del alfabeto latino, _{IsLatin}_ o del conjunto de caracteres comunes, _{IsCommon}_.

In [22]:
for name in names:
    if not contains_valid_characters(name):
        print(name)

❤️
❤️
☁️☁️☁️
❄️
✔️
( ͡° ͜ʖ ͡°)
⚡️⚡️⚡️
⚡️
❄️
🏃🏼‍♀️
‼️
❤️
❤️❤️
❤️
❤️


Vemos que aquellos títulos que la función nos ha devuelto como _False_ contienen solamente emoticonos (a excepción del 2 y el 11). Por lo tanto, consideramos que no tenemos otros caracteres pertenecientes a otros alfabetos.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nuevo criterio__: Los títulos sólo contienen caracteres del alfabeto latino y emoticonos.
</div>

---

### <font color="#B20033" size=3>Otras observaciones</font>

<br>

Para hacernos una idea de qué contienen los títulos de las playlists, vamos a ver cuáles son las palabras (junto al número de apariciones) que existen en el conjunto:

In [23]:
word_list = Counter(' '.join(names).lower().split()).most_common()
word_list[0:15]

[('country', 149),
 ('chill', 117),
 ('music', 106),
 ('songs', 94),
 ('summer', 89),
 ('rock', 77),
 ('new', 71),
 ('playlist', 66),
 ('good', 64),
 ('party', 60),
 ('rap', 60),
 ('workout', 59),
 ('the', 58),
 ('christmas', 55),
 ('mix', 50)]

Como podemos observar, algunas de las palabras más frecuentes que contienen los títulos son:
- Géneros musicales
- Estaciones del año
- Estados de ánimo
- Actividades
- Eventos
- ...

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section6"></a>
## <font color="#92002A">6 - Conclusiones</font>
<br>

Una vez estudiada la muestra obtenida del *Million Playlist Dataset*, hemos obtenido los siguientes criterios para crear nuestro propio conjunto:

1. La playlist es pública (al menos durante la fase de descarga).
2. Contiene entre 5 y 250 pistas.
3. Contiene al menos 3 artistas únicos.
4. Contiene al menos 2 álbumes únicos.
5. No contiene pistas locales.
6. Tienen al menos un seguidor.
7. No tiene un título ofensivo.
8. El título contiene entre 2 y 50 caracteres (sin espacios).
9. El título contiene menos de 10 palabras.
10. Los títulos sólo contienen caracteres del alfabeto latino y emoticonos.
11. El título contiene como máximo 10 emoticonos, si también contiene texto.
12. Si el título está formado únicamente por emoticonos, contiene un número máximo de 4.

<br>

La mayor parte de estos criterios serán aplicados en la libreta [_Filtrado de Playlists_](04-FiltradoPlaylists.ipynb). Sin embargo, para reducir el número de playlists a obtener y facilitar la descarga, otros de ellos serán aplicados en las siguientes libretas:
- [Preparación de la descarga de playlists](02-PreparacionDescarga.ipynb).
- [Descarga de playlists](03-DescargaPlaylists.ipynb).

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-graduation-cap" aria-hidden="true" style="color:#92002A"></i> </font></div>